In [33]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time
import datetime
import xlsxwriter
import circuit_builder
import importlib
importlib.reload(circuit_builder)


#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

backend = 'local_qasm_simulator'

def permute(gateOrder,perm):
    permCopy = perm.copy()
    for i in range(len(perm)):
        print(i)
        print(gateOrder[i])
        perm[i+1] = permCopy[int(gateOrder[i])]
            

# fill this in later  
# def writeToExcel():
    
    
# all the important settings are here    
shots = 1024
shouldWrite = True

# maps the input info into actual physical qubits
perm = {1: 1, 2:0}

# all circuits we want to test
circuits = [["permute 2 1", "x 1", "h 2", "cx 2 1"]]

# fill these in later to get a calculation of success rate
desiredOutputs = ["10", "01"]
validOutputs = ["00", "01", "10", "11"]

# main benchmarking procedure
for circuit in circuits:
    n = 2
    qr = QuantumRegister(n)
    cr = ClassicalRegister(n)
    qc = QuantumCircuit(qr,cr)
    print(circuit)
    
    # check if there's a permutation
    first = circuit[0].split(" ")
    if first[0] == "permute":
        permute(first[1:], perm)
        circuit = circuit[1:]
    print("got here")

    circuit_builder.buildCirc(qc, qr, circuit, perm)

    for i in range(n):
        qc.measure(qr[perm[i+1]], cr[i])


    job = execute(qc, backend,shots=shots)
    result = job.result()
    data = result.get_counts(qc)
    
    print(data)
    successes = 0
    totalValid = 0
    for key, value in data.items():
        if key in desiredOutputs:
            successes += value
        if key in validOutputs:
            totalValid += value
    
    if totalValid == 0:
        print("oops")
    else:
        print("Success rate is: "+ str(successes/totalValid))


Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\qecc_benchmark\Qconfig.py.
['permute 2 1', 'x 1', 'h 2', 'cx 2 1']
0
2
1
1
got here
x 1
h 2
cx 2 1
{'01': 521, '10': 503}
Success rate is: 1.0
